# 스마트 음성인식을 위한 소스코드


## 학습을 위한 기본 설정


os.environ 은 학습의 효율성을 증대시키기 위한 코드입니다.
learning rate 및 학습 횟수를 지정하여 학습의 시간을 줄여줍니다.

In [0]:
import os

# 학습하려고 하는 단어를 미리 설정하여 학습 시간을 단축하기 위한 코드
os.environ["WANTED_WORDS"] = "on, off"

# 학습의 횟수는 15000 + 3000 으로 총 18,000번 수행
# Learning rate는 0.001, 0.0001 으로 설정
os.environ["TRAINING_STEPS"]="15000,3000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# 현재 설정된 값을 echo 명령어를 통해 출력
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"


## 학습을 위한 라이브러리 설치

텐서플로우와 텐서보드를 설치합니다.

In [0]:
!pip uninstall -y tensorflow tensorflow_estimator tensorboard
!pip install -q tf-estimator-nightly==1.14.0.dev2019072901 tf-nightly-gpu==1.15.0.dev20190729

Github를 통해 공개된 텐서플로우 소스코드를 설치합니다.

In [0]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout 17ce384df70

## 텐서보드를 설치하여 이를 활용해보고자 합니다.

이번 실습에서는 텐서보드를 사용하여 학습의 과정을 그래프의 형태로 확인해봅니다.

In [0]:
# 이전에 가지고 있던 로그는 지우고 텐서보드를 실행합니다.
!rm -rf /content/retrain_logs

%load_ext tensorboard
%tensorboard --logdir /content/retrain_logs

## 학습 시작

스크립트의 형태로 기존에 실습의 형태로 제공되고 있는 코드를 통해 학습을 수행합니다.

In [0]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=25 --unknown_percentage=25 \
--quantize=1 --verbosity=WARN --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=/content/retrain_logs \
--data_dir=/content/speech_dataset --train_dir=/content/speech_commands_train \


## Freeze the graph

Once training is complete, run the following cell to freeze the graph.

학습이 완료된 후에는 Freeze the graph라고 하는 작업을 수행합니다.
Freeze the graph란 학습 모델을 .pb 라고하는 파일로 생성하는 과정을 뜻합니다.
pb 파일을 만듦으로써 C++ 기반의 텐서플로우 Lite에서 해당 학습모델을 사용할 수 있습니다.

In [0]:
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=/content/tiny_conv.pb \
--start_checkpoint=/content/speech_commands_train/tiny_conv.ckpt-${TOTAL_STEPS}


## 모델의 형태 변경

이전 실습과 마찬가지로 텐서플로우에서 학습된 모델을 텐서플로우 Lite에서 사용할 수 있는 모델로 변경합니다.

In [0]:
!toco \
--graph_def_file=/content/tiny_conv.pb --output_file=/content/tiny_conv.tflite \
--input_shapes=1,49,40,1 --input_arrays=Reshape_2 --output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

텐서플로우 Lite에서 사용 가능한 모델을 아두이노 IDE에서 사용하기 위해 .cc 파일의 형태로 저장합니다.

xxd 패키지를 설치하여 tflite 파일을 cc 파일로 저장합니다.

In [0]:

!apt-get -qq install xxd

!xxd -i /content/tiny_conv.tflite > /content/tiny_conv.cc

!cat /content/tiny_conv.cc